In [ ]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
from dotenv import load_dotenv
import glob
from IPython.display import display

In [ ]:
# 1. Defina o caminho e o padrão de busca
CAMINHO_SAIDAS_ETL = r"C:\Users\jea_goncalves\Desktop\balancetes\saidas\pkl"
padrao_magalu_pkl = "bal_magalu_processado_*.pkl"
caminho_busca_magalu = os.path.join(CAMINHO_SAIDAS_ETL, padrao_magalu_pkl)

# 2. Encontra todos os arquivos que correspondem ao padrão
arquivos_magalu_encontrados = glob.glob(caminho_busca_magalu)

# 3. Carrega o arquivo mais recente
if arquivos_magalu_encontrados:
    # Encontra o caminho do arquivo com a data de modificação mais recente
    caminho_recente_magalu = max(arquivos_magalu_encontrados, key=os.path.getmtime)
    print(f"Carregando arquivo encontrado: {os.path.basename(caminho_recente_magalu)}")
    
    # Carrega o DataFrame do arquivo pickle
    df_magalu = pd.read_pickle(caminho_recente_magalu)
    
    print("DataFrame 'df_magalu' carregado com sucesso.")
    # print(df_magalu.info()) # Descomente para verificar os tipos de dados
else:
    print(f"ERRO: Nenhum arquivo encontrado com o padrão '{padrao_magalu_pkl}'")
    df_magalu = pd.DataFrame() # Cria um DataFrame vazio para evitar erros futuros


In [ ]:
# Célula 4: Carregando os Balancetes da Época (.pkl)

# (Mesma função que você usa para salvar os arquivos)

def obter_strings_de_mes():
    hoje = datetime.now()
    mes_atual = hoje.strftime("%m-%y")
    
    mes_anterior_data = hoje - relativedelta(months=1)
    mes_anterior = mes_anterior_data.strftime("%m-%y")
    
    return mes_atual, mes_anterior

# Obtém as strings que serão usadas nos nomes dos arquivos
mes_atual_str, mes_anterior_str = obter_strings_de_mes()

# Assumindo que CAMINHO_SAIDAS_ETL é a variável que aponta para a pasta 'pkl'
# Se não for, ajuste para CAMINHO_DE_SAIDA_PKL do seu outro script
CAMINHO_SAIDAS_ETL = r"C:\Users\jea_goncalves\Desktop\balancetes\saidas\pkl"


# --- ETAPA 2: CARREGAR BALANCETE ÉPOCA (MÊS ANTERIOR) USANDO O PADRÃO DINÂMICO ---

# AJUSTE: O padrão agora usa a string 'mes_anterior_str'
padrao_epoca_anterior_pkl = f"bal_epoca_{mes_anterior_str}_*.pkl"
caminho_busca_epoca_ant = os.path.join(CAMINHO_SAIDAS_ETL, padrao_epoca_anterior_pkl)
arquivos_epoca_ant_encontrados = glob.glob(caminho_busca_epoca_ant)

if arquivos_epoca_ant_encontrados:
    caminho_recente_epoca_ant = max(arquivos_epoca_ant_encontrados, key=os.path.getmtime)
    print(f"Carregando arquivo do mês anterior: {os.path.basename(caminho_recente_epoca_ant)}")
    df_epoca_anterior = pd.read_pickle(caminho_recente_epoca_ant)
    print("DataFrame 'df_epoca_anterior' carregado com sucesso.\n")
else:
    print(f"AVISO: Nenhum arquivo encontrado para o Mês Anterior com o padrão '{padrao_epoca_anterior_pkl}'")
    df_epoca_anterior = pd.DataFrame() # Cria um DataFrame vazio para não gerar erro


# --- ETAPA 3: CARREGAR BALANCETE ÉPOCA (MÊS ATUAL) USANDO O PADRÃO DINÂMICO ---

# AJUSTE: O padrão agora usa a string 'mes_atual_str'
padrao_epoca_atual_pkl = f"bal_epoca_{mes_atual_str}_*.pkl"
caminho_busca_epoca_atual = os.path.join(CAMINHO_SAIDAS_ETL, padrao_epoca_atual_pkl)
arquivos_epoca_atual_encontrados = glob.glob(caminho_busca_epoca_atual)

if arquivos_epoca_atual_encontrados:
    caminho_recente_epoca_atual = max(arquivos_epoca_atual_encontrados, key=os.path.getmtime)
    print(f"Carregando arquivo do mês atual: {os.path.basename(caminho_recente_epoca_atual)}")
    df_epoca_atual = pd.read_pickle(caminho_recente_epoca_atual)
    print("DataFrame 'df_epoca_atual' carregado com sucesso.")
else:
    print(f"AVISO: Nenhum arquivo encontrado para o Mês Atual com o padrão '{padrao_epoca_atual_pkl}'")
    df_epoca_atual = pd.DataFrame() # Cria um DataFrame vazio para não gerar erro


In [ ]:
load_dotenv()
CAMINHO_CREDENCIAL_JSON = os.getenv('GOOGLE_CREDENTIALS_SERVICE_ACCOUNT')
ID_PLANILHA = os.getenv('GOOGLE_SHEET_ID')
NOME_ABA  = os.getenv('WORKSHEET_NAME')
TOKEN_USER = os.getenv('AUTHORIZED_USER_FILE_PATH')
gc = gspread.service_account(
    filename=CAMINHO_CREDENCIAL_JSON)

In [ ]:
ws = gc.open_by_key(ID_PLANILHA)
aba = ws.worksheet(NOME_ABA)
data = aba.get_all_values(value_render_option='FORMATTED_VALUE')
header, rows = data[0], data[1:]
df = pd.DataFrame(rows, columns=header)
df.head(100)

In [ ]:
def convert_br_numbers(df, cols):
    for col in cols:
        df[col] = (
            df[col]
            .astype(str)                              # força string
            .str.replace(".", "", regex=False)        # remove separador de milhar
            .str.replace(",", ".", regex=False)     # troca vírgula por ponto
            .replace("nan", None)                      # limpa 'nan' como string
            .replace("", None)                         # trata células vazias
            .astype(float)                             # converte para float
        )
    return df


In [ ]:
num_cols = ["SALDO_INICIAL", "DEBITO", "CREDITO", "SALDO_FINAL", "MOV_MES"]

df = convert_br_numbers(df, num_cols)

pd.set_option("display.float_format", "{:,.2f}".format)

df.head()

In [ ]:
print("\nCriando backup da planilha antes da atualização...")
# Usamos get_all_values para uma cópia fiel da estrutura e dados
backup_data = aba.get_all_values()
if backup_data:
    df_backup = pd.DataFrame(backup_data[1:], columns=backup_data[0])
    print("Backup criado com sucesso na variável 'df_backup'.")
else:
    # Se a planilha estiver vazia, cria um backup vazio para evitar erros.
    df_backup = pd.DataFrame()
    print("A planilha está vazia. Um backup vazio foi criado.")

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def obter_strings_de_mes():
    """Calcula dinamicamente as strings para o mês atual e anterior no formato 'MM-YY'."""
    hoje = datetime.now()
    mes_atual = hoje.strftime("%m-%y")
    
    mes_anterior_data = hoje - relativedelta(months=1)
    mes_anterior = mes_anterior_data.strftime("%m-%y")
    
    return mes_atual, mes_anterior

def encontrar_arquivo_recente(padrao):
    """Encontra o arquivo mais recente que corresponde a um padrão de nome."""
    try:
        lista_arquivos = glob.glob(padrao)
        if not lista_arquivos:
            print(f"Aviso: Nenhum arquivo encontrado para o padrão: {padrao}")
            return None
        arquivo_mais_recente = max(lista_arquivos, key=os.path.getctime)
        return arquivo_mais_recente
    except Exception as e:
        print(f"Erro ao buscar arquivo com padrão '{padrao}': {e}")
        return None

# Obtendo os meses
mes_atual_str, mes_anterior_str = obter_strings_de_mes()
print(f"Mês Atual: {mes_atual_str}, Mês Anterior: {mes_anterior_str}")

In [ ]:
# Pré-requisito: A célula que define 'mes_atual_str' e 'mes_anterior_str' DEVE ser executada antes desta.

print(f"Usando os meses já definidos: Atual ({mes_atual_str}) e Anterior ({mes_anterior_str})")

# --- CARREGAR ARQUIVOS .PKL PROCESSADOS PELO ETL ---

# Defina o diretório onde os arquivos .pkl foram salvos
CAMINHO_SAIDAS_ETL = r"C:\Users\jea_goncalves\Desktop\balancetes\saidas\pkl"

# 1. Carregar Balancete Magalu
padrao_magalu_pkl = "bal_magalu_processado_*.pkl"
caminho_busca_magalu = os.path.join(CAMINHO_SAIDAS_ETL, padrao_magalu_pkl)
arquivos_magalu_encontrados = glob.glob(caminho_busca_magalu)

if arquivos_magalu_encontrados:
    caminho_recente_magalu = max(arquivos_magalu_encontrados, key=os.path.getmtime)
    print(f"\nCarregando arquivo Magalu: {os.path.basename(caminho_recente_magalu)}")
    df_magalu = pd.read_pickle(caminho_recente_magalu)
else:
    print(f"\nAVISO: Nenhum arquivo do Magalu encontrado com o padrão '{padrao_magalu_pkl}'")
    df_magalu = pd.DataFrame()

# 2. Carregar Balancete Época (Mês Anterior)
padrao_epoca_anterior_pkl = f"bal_epoca_{mes_anterior_str}_*.pkl"
caminho_busca_epoca_ant = os.path.join(CAMINHO_SAIDAS_ETL, padrao_epoca_anterior_pkl)
arquivos_epoca_ant_encontrados = glob.glob(caminho_busca_epoca_ant)

if arquivos_epoca_ant_encontrados:
    caminho_recente_epoca_ant = max(arquivos_epoca_ant_encontrados, key=os.path.getmtime)
    print(f"Carregando arquivo Época (Mês Anterior): {os.path.basename(caminho_recente_epoca_ant)}")
    df_epoca_anterior = pd.read_pickle(caminho_recente_epoca_ant)
else:
    print(f"AVISO: Nenhum arquivo da Época (Mês Anterior) encontrado com o padrão '{padrao_epoca_anterior_pkl}'")
    df_epoca_anterior = pd.DataFrame()

# 3. Carregar Balancete Época (Mês Atual)
padrao_epoca_atual_pkl = f"bal_epoca_{mes_atual_str}_*.pkl"
caminho_busca_epoca_atual = os.path.join(CAMINHO_SAIDAS_ETL, padrao_epoca_atual_pkl)
arquivos_epoca_atual_encontrados = glob.glob(caminho_busca_epoca_atual)

if arquivos_epoca_atual_encontrados:
    caminho_recente_epoca_atual = max(arquivos_epoca_atual_encontrados, key=os.path.getmtime)
    print(f"Carregando arquivo Época (Mês Atual): {os.path.basename(caminho_recente_epoca_atual)}")
    df_epoca_atual = pd.read_pickle(caminho_recente_epoca_atual)
else:
    print(f"AVISO: Nenhum arquivo da Época (Mês Atual) encontrado com o padrão '{padrao_epoca_atual_pkl}'")
    df_epoca_atual = pd.DataFrame()

In [ ]:
from datetime import datetime # Certifique-se de que datetime está importado

def atualizar_secao_balancete(df, df_origem, mes_str, tipo_empresa):
    """
    Função principal para atualizar uma seção do balancete na planilha.
    Versão final com o cálculo correto da data_efetiva para a Época.
    """
    # Trava de segurança para não processar meses muito antigos
    if mes_str not in [mes_atual_str, mes_anterior_str]:
        print(f"\n--- AVISO: O período '{mes_str}' está fora da janela de atualização. Pulando o processamento para '{tipo_empresa}'.")
        return df

    print(f"\n--- Iniciando atualização para: {tipo_empresa.upper()} | Mês: {mes_str} ---")
    
    if df_origem.empty:
        print("Aviso: DataFrame de origem está vazio. Pulando esta atualização.")
        return df

    df_fonte = df_origem.copy()
    df_atualizado = df.copy()

    # 1. Padroniza os nomes das colunas da fonte
    if tipo_empresa == 'magalu':
        mapa_colunas = {'periodo': 'periodo', 'empresa': 'empresa', 'conta': 'conta', 'descricao_conta': 'descricao_conta', 'saldo_inicial': 'saldo_inicial', 'debito': 'debito', 'credito': 'credito', 'saldo_final': 'saldo_final'}
    else: # epoca
        mapa_colunas = {'Classificação': 'conta', 'Nome': 'descricao_conta', 'Saldo Anterior': 'saldo_inicial', 'Débito': 'debito', 'Crédito': 'credito', 'Saldo Atual': 'saldo_final'}
    
    df_fonte = df_fonte.rename(columns=mapa_colunas)
    if 'empresa' not in df_fonte.columns: df_fonte['empresa'] = '2'
    if 'periodo' not in df_fonte.columns: df_fonte['periodo'] = mes_str
    df_fonte['empresa'] = df_fonte['empresa'].astype(str)

    # 2. Filtra o bloco de destino e ajusta as linhas (lógica mantida)
    if tipo_empresa == 'magalu':
        filtro_df = (df_atualizado['periodo'] == mes_str) & (df_atualizado['empresa'] != '2')
    else:
        filtro_df = (df_atualizado['periodo'] == mes_str) & (df_atualizado['empresa'] == '2')
    df_bloco_original = df_atualizado[filtro_df]

    ponto_de_insercao = -1
    if not df_bloco_original.empty:
        ponto_de_insercao = df_bloco_original.index[-1]
    else:
        if tipo_empresa == 'epoca':
            filtro_magalu_irmao = (df_atualizado['periodo'] == mes_str) & (df_atualizado['empresa'] != '2')
            bloco_magalu_irmao = df_atualizado[filtro_magalu_irmao]
            if not bloco_magalu_irmao.empty:
                ponto_de_insercao = bloco_magalu_irmao.index[-1]
        if ponto_de_insercao == -1:
            registros_anteriores = df_atualizado[df_atualizado['periodo'] < mes_str]
            if not registros_anteriores.empty:
                ponto_de_insercao = registros_anteriores.index.max()
    
    n_linhas_fonte = len(df_fonte)
    diferenca = n_linhas_fonte - len(df_bloco_original)

    if diferenca > 0:
        novas_linhas_df = pd.DataFrame([{} for _ in range(diferenca)], columns=df_atualizado.columns)
        df_antes = df_atualizado.loc[:ponto_de_insercao]
        df_depois = df_atualizado.loc[ponto_de_insercao+1:]
        df_atualizado = pd.concat([df_antes, novas_linhas_df, df_depois], ignore_index=True)
    elif diferenca < 0:
        indices_para_remover = df_bloco_original.index[n_linhas_fonte:]
        df_atualizado = df_atualizado.drop(indices_para_remover).reset_index(drop=True)

    # 3. Colagem final
    if n_linhas_fonte > 0:
        idx_inicio = df_bloco_original.index[0] if not df_bloco_original.empty else ponto_de_insercao + 1
        indices_destino = range(idx_inicio, idx_inicio + n_linhas_fonte)
        
        colunas_para_colar = [col for col in df_fonte.columns if col in df_atualizado.columns]
        df_fonte.index = indices_destino
        df_atualizado.loc[indices_destino, colunas_para_colar] = df_fonte[colunas_para_colar]
    
        # ==============================================================================
        # 4. PREENCHIMENTO DE DADOS E FÓRMULAS
        # ==============================================================================
        
        # Pre-calcula a data efetiva para o bloco da Época, se aplicável
        data_efetiva_formatada = ""
        if tipo_empresa == 'epoca':
            try:
                # Converte 'MM-YY' para um objeto data e depois para o formato 'DD/MM/YYYY'
                data_obj = datetime.strptime(f"01-{mes_str}", "%d-%m-%y")
                data_efetiva_formatada = data_obj.strftime("%d/%m/%Y")
            except ValueError:
                print(f"AVISO: Não foi possível formatar a data para o período '{mes_str}'.")

        for idx in indices_destino:
            df_atualizado.loc[idx, 'concatenação'] = f"=CONCATENATE(E{idx+2}, G{idx+2}, B{idx+2})"
            
            if tipo_empresa == 'epoca':
                df_atualizado.loc[idx, 'empresa'] = '2'
                df_atualizado.loc[idx, 'periodo'] = mes_str
                
                if 'mov_mes' in df_atualizado.columns:
                    df_atualizado.loc[idx, 'mov_mes'] = f"=J{idx+2}-K{idx+2}"
                
                # Preenche a data efetiva com o valor calculado
                if 'data_efetiva' in df_atualizado.columns and data_efetiva_formatada:
                    df_atualizado.loc[idx, 'data_efetiva'] = data_efetiva_formatada
                
                # Copia o 'livro' da linha de cima, pois não vem na origem
                if idx > 0 and 'livro' in df_atualizado.columns:
                    df_atualizado.loc[idx, 'livro'] = df_atualizado.loc[idx-1, 'livro']

    print("Atualização concluída.")
    return df_atualizado

In [ ]:
# --- ORDEM DE EXECUÇÃO DAS ATUALIZAÇÕES ---

# Verifica se o DataFrame principal 'df' foi carregado antes de continuar.
if 'df' in locals() and not df.empty:
    print("Iniciando o processo de atualização da planilha...")

    # Começa com uma cópia do DataFrame original do Google Sheets
    df_atualizado = df.copy()

    # --- AJUSTE: Garante que as colunas dos DataFrames estejam em minúsculas ---
    # Isso evita erros de KeyError se as colunas vierem com letras maiúsculas
    # tanto do Google Sheets (df_atualizado) quanto do Oracle (df_magalu).
    df_atualizado.columns = [col.lower() for col in df_atualizado.columns]
    
    if not df_magalu.empty:
        df_magalu.columns = [col.lower() for col in df_magalu.columns]


    # 1. Magalu - Mês Anterior
    # Filtra o df_magalu para passar apenas os dados do mês anterior
    df_magalu_anterior = df_magalu[df_magalu['periodo'] == mes_anterior_str]
    df_atualizado = atualizar_secao_balancete(df_atualizado, df_magalu_anterior, mes_anterior_str, 'magalu')

    # 2. Época - Mês Anterior
    df_atualizado = atualizar_secao_balancete(df_atualizado, df_epoca_anterior, mes_anterior_str, 'epoca')

    # 3. Magalu - Mês Atual
    # Filtra o df_magalu para passar apenas os dados do mês atual
    df_magalu_atual = df_magalu[df_magalu['periodo'] == mes_atual_str]
    df_atualizado = atualizar_secao_balancete(df_atualizado, df_magalu_atual, mes_atual_str, 'magalu')

    # 4. Época - Mês Atual
    df_atualizado = atualizar_secao_balancete(df_atualizado, df_epoca_atual, mes_atual_str, 'epoca')

    print("\n-------------------------------------------")
    print("Processo de atualização finalizado.")
    print("Pré-visualização do DataFrame final antes de enviar para o Google Sheets:")
    display(df_atualizado.head())
    display(df_atualizado.tail())

else:
    print("ERRO: O DataFrame 'df' não foi carregado corretamente. A execução foi interrompida.")

In [ ]:
# CÉLULA DE ATUALIZAÇÃO FINAL (COM AJUSTE DE FÓRMULA)

# O resultado final de tudo isso deve ser o DataFrame 'df_atualizado'

# --- AJUSTE DA FÓRMULA DE EMPRESA/CONTA (COLUNA A) ---
print("\nAjustando a fórmula da primeira coluna para o formato do Google Sheets (Português)...")

if not df_atualizado.empty:
    # Identifica o nome da primeira coluna do DataFrame para substituí-la
    primeira_coluna = df_atualizado.columns[0]
    
    # Gera a lista de fórmulas, começando na linha 2 da planilha
    # (índice 0 do df + 2 = linha 2)
    formulas = [f'=CONCATENAR(F{i};\"/\";G{i})' for i in range(2, len(df_atualizado) + 2)]
    
    # Atribui as novas fórmulas à primeira coluna
    df_atualizado[primeira_coluna] = formulas

    print(f"Fórmula da primeira linha ajustada para: {df_atualizado[primeira_coluna].iloc[0]}")
else:
    print("AVISO: DataFrame 'df_atualizado' está vazio. Nenhuma fórmula foi criada.")

# --- CONVERTE COLUNAS PARA MAIÚSCULAS ---
print("\nConvertendo os nomes das colunas para MAIÚSCULAS antes do envio...")
if not df_atualizado.empty:
    df_atualizado.columns = [col.upper() for col in df_atualizado.columns]
    print("Nomes das colunas convertidos com sucesso.")

# --- ESCRITA FINAL NO GOOGLE SHEETS ---
print("\nIniciando a atualização final da planilha no Google Sheets...")

try:
    aba.clear()
    set_with_dataframe(aba, df_atualizado, include_index=False, resize=True, allow_formulas=True)
    print("\nPlanilha atualizada com sucesso!")
    print("O processo de automação foi concluído.")

except Exception as e:
    print(f"\nOcorreu um erro ao tentar atualizar o Google Sheets: {e}")

#================BACKUP CASO A COLAGEM DER ERRADO=============================

if 'df_backup' in locals():
    print("Iniciando a restauração do backup...")
    try:
        # Garante que a conexão com a worksheet ainda existe
        if 'worksheet' not in locals():
             print("Reconectando à planilha para restauração...")
             spreadsheet = gc.open(ws)
             worksheet = spreadsheet.worksheet(aba)

        worksheet.clear()
        set_with_dataframe(worksheet, df_backup, include_index=False, resize=True, allow_formulas=True)
        print("\nSUCESSO: Planilha restaurada para o estado anterior à última execução.")
    except Exception as e:
        print(f"Ocorreu um erro durante a restauração: {e}")
else:
    print("ERRO: Nenhum backup (df_backup) foi encontrado na memória. Não é possível restaurar.")